In [ ]:
import pandas as pd
import seaborn as sns

import ast
from functools import partial

In [ ]:
data = pd.read_csv(r"../data/raw/ptbxl_database.csv", index_col="ecg_id")

In [ ]:
data.head()

In [ ]:
data.columns

In [ ]:
unique_diagnosis = data.scp_codes.unique()
unique_diagnosis, len(unique_diagnosis)

In [ ]:
data.scp_codes = data.scp_codes.apply(ast.literal_eval)

In [ ]:
data.scp_codes

In [ ]:
scp_statements = pd.read_csv(r"../data/raw/scp_statements.csv", index_col=0)

In [ ]:
scp_statements.head()

In [ ]:
scp_statements.columns

In [ ]:
scp_statements.diagnostic_class.unique()

In [ ]:
scp_superclass = scp_statements.diagnostic_class
scp_superclass

In [ ]:
def probs_to_list(threshold: int, probs: dict[str, int]) -> tuple[str]:
    result = [key for key, value in probs.items() if value >= threshold]
    
    return tuple(result)

In [ ]:
test_scp_probs  = data.scp_codes.iloc[0]
test_scp_probs

In [ ]:
probs_to_list(50, test_scp_probs)

In [ ]:
probs_to_list_15 = partial(probs_to_list, 15)

In [ ]:
data["diagnoses"] = data.scp_codes.apply(probs_to_list_15)
data.diagnoses

In [ ]:
data.diagnoses.unique()

In [ ]:
len(data[data.diagnoses == tuple()].scp_codes)

In [ ]:
data.diagnoses

In [ ]:
scp_statements = scp_statements[scp_statements.diagnostic == 1]
scp_statements

In [ ]:
scp_superclass = scp_statements.diagnostic_class
scp_superclass

In [ ]:
def diagnosis_to_superclass(diag_sp_map: pd.Series, diags: tuple[str]) -> tuple[str]:
    superclasses = set()
    for diag in diags:
        try:
            superclass = diag_sp_map[diag]
        except KeyError:
            continue
        
        superclasses.add(superclass)

    return tuple(superclasses)

In [ ]:
diagnosis_to_superclass(scp_superclass, ("NDT", "PVC", "DIG")) # Some tests passed

In [ ]:
diagnosis_to_superclass_sp = partial(diagnosis_to_superclass, scp_superclass)

In [ ]:
data["superclass"] = data.diagnoses.apply(diagnosis_to_superclass_sp)
data.superclass

In [ ]:
data.superclass.unique()

In [ ]:
len(data[(data.superclass == tuple())])

In [ ]:
data["NORM" in data.superclass]